A planner is a potent function that utilizes AI to combine registered plugins within the kernel and generate a plan to fulfill a user's request. This powerful concept empowers developers to create atomic functions, enabling them to construct complex workflows without explicit coding for each scenario. However, it requires caution as it can combine functions in unexpected ways. Adhering to responsible AI principles ensures fair, reliable, and secure usage. The planner is an extensible part of the Semantic Kernel, offering various options, and developers can create custom planners to suit specific needs. Language support is indicated in a table, encouraging contributions to enhance functionality.

## Kernel Instansiation

In [29]:
#r "nuget: Microsoft.SemanticKernel, 0.17.230626.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;
using System;
using System.Linq;
using System.Reflection;

var builder = new KernelBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.WithAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    builder.WithOpenAITextCompletionService(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 0.17.230626.1-preview

Having the planner at our disposal allows us to devise a plan for tackling a user's request and subsequently execute the plan to obtain the desired outcome. The following code demonstrates how we employ the planner to solve a complex math problem, one that proves challenging for an LLM (Large Language Model) to handle independently due to its multi-step nature and the presence of numerical values.

In [30]:
// Load native skill into the kernel registry, sharing its functions with prompt templates
var planner = new SequentialPlanner(kernel);


## Loading Available Skills

In [31]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "CodingSkill");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

## Convert User Ask to a Plan

In [35]:
var ask = " write a pragraph about texas, and use the last sentence as starter of a joke  ";
var originalPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(JsonSerializer.Serialize(originalPlan, new JsonSerializerOptions { WriteIndented = true }));

Error: Microsoft.SemanticKernel.Planning.PlanningException: Create plan error: Unable to create plan
 ---> Microsoft.SemanticKernel.Planning.PlanningException: Invalid plan: Failed to find function 'Entity' in skill 'Entity'.
   at Microsoft.SemanticKernel.Planning.Sequential.SequentialPlanParser.ToPlanFromXml(String xmlString, String goal, SKContext context, Boolean allowMissingFunctions)
   at Microsoft.SemanticKernel.Planning.SequentialPlanner.CreatePlanAsync(String goal)
   --- End of inner exception stack trace ---
   at Microsoft.SemanticKernel.Planning.SequentialPlanner.CreatePlanAsync(String goal)
   at Submission#42.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Execute the Plan

In [27]:
var originalPlanResult = await originalPlan.InvokeAsync();

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(originalPlanResult.Result);

Original Plan results:

in the format of "dd/mm/yyyy"
def printDate(date):
    #Split the date into day, month and year
    day, month, year = date.split('/')
    #Print the date in the required format
    print(f"{day}/{month}/{year}")
#Done


In [13]:
var newPlanResult = await kernel.RunAsync(newPlan);
Console.WriteLine("New Plan results:\n");
Console.WriteLine(newPlanResult.Result);

Error: (1,43): error CS0103: The name 'newPlan' does not exist in the current context